In [ ]:
# 체인 연결하기: 복합 AI 워크플로우의 구현
# 1단계: 셰프가 레시피 생성 → 2단계: 베지테리안 셰프가 채식 버전으로 변환

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# 스트리밍 설정: 응답을 실시간으로 볼 수 있음
chat = ChatOpenAI(
    temperature=0.1,        # 일관된 답변을 위한 낮은 창의성
    streaming=True,         # 실시간 스트리밍 활성화
    callbacks=[
        # 콘솔에 실시간으로 출력하는 콜백
        # 응답이 생성되는 과정을 실시간으로 관찰 가능
        StreamingStdOutCallbackHandler(),
    ],
)

# 1단계: 일반 셰프 프롬프트 (레시피 생성 전문)
chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 비어있는 시스템 메시지 - 실제 강의에서는 내용이 있었을 것
            "",
        ),
        # 요리 스타일을 입력받아 레시피 생성 요청
        ("human", "I want to cook {cuisine} food."),
    ]
)

# 첫 번째 체인: 셰프 프롬프트 + Chat 모델
chef_chain = chef_prompt | chat

In [ ]:
# 2단계: 베지테리안 전문 셰프 + 최종 체인 구성
# 핵심: 첫 번째 체인의 출력을 두 번째 체인의 입력으로 연결

# 베지테리안 셰프 프롬프트 (채식 변환 전문)
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # 베지테리안 셰프의 역할 정의:
            # 1. 전통 레시피를 채식으로 변환하는 전문가
            # 2. 대체 재료를 찾아 설명
            # 3. 레시피를 크게 변경하지 않음
            # 4. 대안이 없으면 솔직하게 모른다고 말함 (환각 방지)
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        # 첫 번째 체인에서 생성된 레시피를 입력받음
        ("human", "{recipe}"),
    ]
)

# 두 번째 체인: 베지테리안 프롬프트 + Chat 모델
veg_chain = veg_chef_prompt | chat

# 🎯 핵심! RunnableMap을 이용한 체인 연결
# {"recipe": chef_chain}: chef_chain의 출력을 "recipe" 키로 매핑
# 즉, chef_chain 실행 결과가 veg_chain의 {recipe} 변수로 전달됨
final_chain = {"recipe": chef_chain} | veg_chain

# 실행 순서:
# 1. {"cuisine": "indian"} → chef_chain 입력
# 2. chef_chain 실행 → 인도 요리 레시피 생성
# 3. 레시피를 {"recipe": "생성된 레시피..."} 형태로 매핑
# 4. veg_chain으로 전달 → 채식 버전으로 변환
# 
# 🔥 중요: 변수명 "recipe"는 veg_chef_prompt의 {recipe}와 정확히 일치해야 함!
final_chain.invoke({"cuisine": "indian"})